In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import math # For radians in rose plot
from matplotlib.colors import ListedColormap

from IPython.display import display
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from collections import Counter

import random
import subprocess
import json # Import json to load the stats file
import pandas as pd # Import pandas for DataFrame summary
from utilities import plot_fault_counts, plot_histogram, plot_rose_diagram, append_param_to_cmd, normalize, plot_fault_points, count_pixels   
from IPython.display import display
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import shutil

from cigvis.plotlyplot import create_overlay, plot3D   # <-- add this
from cigvis import colormap, config

import cigvis.plotlyplot as cgp
from cigvis.plotlyplot import create_slices, add_mask, plot3D
from cigvis import colormap, config

from tqdm import trange

---

---

# Data Generation

In [2]:
# 0)  User picks parameters ──────────────────────────────────────────────
mask_mode      = 0
num_pairs      = 101
train_split    = 100          # absolute counts
val_split      = 1

cube_size      = 128
ricker_freq    = (50, 70)
seismic_noise  = (0.1, 0.5)
ricker_dt      = 0.002
wavelet_len    = 0.2
num_gauss      = (2, 10)
num_faults     = (0, 4)
max_disp       = (-50, 50)

# ─────────────────────────────────────────────────────────────────────────
# CHOOSE ONE “strike_windows” line below and **comment-out** the others.
# The text must be in quotes -- it is passed to the script verbatim.
# ─────────────────────────────────────────────────────────────────────────
# strike_windows = "0-360"                 # full azimuth
# strike_windows = "40-50"                 # one narrow cone (≈ 45°)
# strike_windows = "40-50,220-230"         # two opposite cones (≈ 45° & 225°)
#strike_windows = "45-135,225-315"          # two half-circles (or "315-45,135-225")
strike_windows = "40-50,220-230,130-140,310-320"
# ─────────────────────────────────────────────────────────────────────────

dip            = (60, 70)
output_format  = "npy"


In [3]:
# explicit counts
num_train = int(train_split)
num_val   = int(val_split)

if num_train + num_val != num_pairs:
    raise ValueError(
        f"ERROR: train+val ({num_train + num_val}) "
        f"does not equal num_pairs ({num_pairs})."
    )
else:
    print(f"✔  Using {num_train} training and {num_val} validation cubes.")

✔  Using 100 training and 1 validation cubes.


In [4]:
# ───────────────────────────────────────────────────────────────
# 1)  ROOT directory – works in .py or notebook
# ───────────────────────────────────────────────────────────────
try:                                   # for plain-.py execution
    ROOT_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:                      # Jupyter / IPython
    ROOT_DIR = os.getcwd()

In [5]:
# ───────────────────────────────────────────────────────────────
# 2)  Centralised output locations  (⇦ NEW)
# ───────────────────────────────────────────────────────────────
DATA_DIR   = os.path.join(ROOT_DIR, "data")        # …/generateSynthData/data
STATS_DIR  = os.path.join(ROOT_DIR, "statistics")  # …/generateSynthData/statistics
IMAGE_DIR  = os.path.join(ROOT_DIR, "images")      # …/generateSynthData/images

# make sure they exist
for _d in (DATA_DIR, STATS_DIR, IMAGE_DIR):
    os.makedirs(_d, exist_ok=True)

In [6]:
def parse_windows(s):
    out = []
    for rng in s.split(','):
        lo, hi = map(lambda x: float(x.strip()), rng.split('-'))
        out.append((lo, hi))
    return out

In [ ]:
# ─── call generator script ───────────────────────────────────────────────
script    = os.path.join(ROOT_DIR, "synthDataGeneration.py")
base_out  = DATA_DIR          # where the cubes will be written

# wipe previous run
for split in ("train", "validation"):
    shutil.rmtree(os.path.join(base_out, split), ignore_errors=True)

os.makedirs(base_out, exist_ok=True)

cmd = [
    "python", script,
    "--num-pairs", str(num_pairs),
    "--size",      str(cube_size),
    "--dt",        str(ricker_dt),
    "--length",    str(wavelet_len),
    "--mask-mode", str(mask_mode),
    "--format",    output_format,
    "--output-dir", base_out,
    "--train-split", str(train_split),
    "--val-split",   str(val_split),

    "--faults",   f"{num_faults[0]},{num_faults[1]}",
    f"--max-disp={max_disp[0]},{max_disp[1]}",

    "--strike",   strike_windows,
    "--dip",      f"{dip[0]},{dip[1]}",
    "--freq",     f"{ricker_freq[0]},{ricker_freq[1]}",
    "--num-gaussians", f"{num_gauss[0]},{num_gauss[1]}",
    "--noise",    f"{seismic_noise[0]},{seismic_noise[1]}",
    "--strike-sampling", "equal" # or "span" where “equal” = each window chosen with equal probability; “span” = proportional to angular width.
]

print(" ".join(cmd))
result = subprocess.run(cmd, text=True)

python /home/roderickperez/DS_PROJECTS/faultSeg/generateSynthData/synthDataGeneration.py --num-pairs 101 --size 128 --dt 0.002 --length 0.2 --mask-mode 0 --format npy --output-dir /home/roderickperez/DS_PROJECTS/faultSeg/generateSynthData/data --train-split 100 --val-split 1 --faults 0,4 --max-disp=-50,50 --strike 40-50,220-230,130-140,310-320 --dip 60,70 --freq 50,70 --num-gaussians 2,10 --noise 0.1,0.5 --strike-sampling equal

Data split: 100 train, 1 validation.
Output format: npy

Starting data generation for 101 pairs...


Generating Cubes:   0%|          | 0/101 [00:00<?, ?it/s]

In [ ]:
if result.returncode == 0:
    print("Data generation successful!")
    print(result.stdout)
else:
    print("Data generation failed.")
    print(result.stderr)

for fname in ("statistics_full.json",
              "statistics_train.json",
              "statistics_validation.json"):
    src = os.path.join(base_out, fname)
    dst = os.path.join(STATS_DIR, fname)
    if os.path.exists(src):
        try:
            os.replace(src, dst)          # move (overwrite if exists)
            print(f"Moved {fname} → {STATS_DIR}")
        except Exception as e:
            print(f"Could not move {fname}: {e}")
    else:
        print(f"Warning: {fname} not found in {base_out}")
        
# Data generation statistics
stats = {
    "total_pairs": num_pairs,
    "train_pairs": num_train,
    "validation_pairs": num_val,
}

# Save statistics to a JSON file
stats_file_path = os.path.join(STATS_DIR, "stats_data.json")
with open(stats_file_path, 'w') as f:
    json.dump(stats, f, indent=4)

print(f"Data generation complete. Statistics saved to {stats_file_path}")

In [ ]:
# ─── 2) Check the output ────────────────────────────────────────────────
# Check the total number of files created across all splits
total_files_generated = 0
for split in ['train', 'validation']:
    seis_dir = os.path.join(base_out, split, 'seis')
    if os.path.exists(seis_dir):
        total_files_generated += len(os.listdir(seis_dir))

if total_files_generated != num_pairs:
    print(f"\nWarning: Mismatch in generated files ({total_files_generated}) and requested pairs ({num_pairs}). Check script output.")
else:
    print(f"\nSuccessfully generated {total_files_generated} file pairs.")

In [ ]:
# Append parameters that can be fixed values or ranges
def append_param_to_cmd(cmd_list, arg_name, value):
    if value is not None:
        if isinstance(value, tuple):
            cmd_list += [f"--{arg_name}", f"{value[0]},{value[1]}"]
        else:
            cmd_list += [f"--{arg_name}", str(value)]

In [ ]:
print("\nCommand to run:")
print(" ".join(cmd)) # type: ignore # Print the corrected command
print("\nRunning subprocess...")

# run and capture output
# check=False allows inspection of returncode without raising exception immediately
#result = subprocess.run(cmd, capture_output=True, text=True, check=False)
print("=== STDOUT ===\n", result.stdout)
print("=== STDERR ===\n", result.stderr)

if result.returncode != 0:
    print(f"Error: Script exited with return code {result.returncode}")
    # Optionally print more context or raise error
    # raise subprocess.CalledProcessError(result.returncode, cmd, output=result.stdout, stderr=result.stderr)
    # Exit this cell or notebook execution if script failed critically?
    # For now, just print error and continue, assuming stats file might still be partially saved
else:
    print("Script finished successfully.")

# Check if the expected number of files were generated
total_files_generated = 0
for split in ['train', 'validation']:
    seis_dir = os.path.join(base_out, split, 'seis')
    if os.path.exists(seis_dir):
        total_files_generated += len([f for f in os.listdir(seis_dir) if f.endswith((".npy", ".npz", ".dat"))])

if total_files_generated != num_pairs:
    print(f"\nWarning: Mismatch in generated files ({total_files_generated}) and requested pairs ({num_pairs}). Check script output.")
else:
    print(f"\nSuccessfully generated {total_files_generated} file pairs across all splits.")

In [ ]:
# ─── 2) Load generated data and statistics ───────────────────────────────────────────────────────

# Define the base output directory
#base_out = os.path.expanduser("/Users/roderickperez/Documents/DS_Projects/faultSegm/faultSeg_Wu_2019_Keras/output")

# Define the splits and their expected number of files
splits = {
    "train": num_train,
    "validation": num_val
}

# Check if the expected number of files were generated for each split
for split, expected_files in splits.items():
    seis_dir = os.path.join(base_out, split, "seis")
    fault_dir = os.path.join(base_out, split, "fault")

    if os.path.exists(seis_dir):
        files = sorted(f for f in os.listdir(seis_dir) if f.endswith((".npy", ".npz", ".dat")))
        n_files = len(files)
        if n_files != expected_files:
            print(f"Warning: Mismatch in generated files for '{split}' split. Expected {expected_files}, found {n_files} in {seis_dir}")
    else:
        print(f"Warning: Seismic output directory not found for '{split}' split: {seis_dir}")

    if not os.path.exists(fault_dir):
        print(f"Warning: Fault output directory not found for '{split}' split: {fault_dir}")

# Load the statistics files
stats_data = {}
for split in splits.keys():
    stats_file_path = os.path.join(STATS_DIR, f"statistics_{split}.json")
    if os.path.exists(stats_file_path):
        with open(stats_file_path, 'r') as f:
            stats_data[split] = json.load(f)
            print(f"Successfully loaded statistics for '{split}' split.")
    else:
        print(f"Warning: Statistics file not found for '{split}' split: {stats_file_path}")

# Load the full statistics file
full_stats_path = os.path.join(STATS_DIR, "statistics_full.json")
if os.path.exists(full_stats_path):
    with open(full_stats_path, 'r') as f:
        full_stats = json.load(f)
        print("Successfully loaded full statistics.")
else:
    print(f"Warning: Full statistics file not found: {full_stats_path}")

In [ ]:
def in_windows(a, windows):
    a = a % 360.0
    for lo, hi in windows:
        lo, hi = lo % 360.0, hi % 360.0
        span = (hi - lo) % 360.0 or 360.0
        if (a - lo) % 360.0 <= span:
            return True
    return False

In [ ]:
# --- Distribution of strikes across windows (full dataset) ---
wins = parse_windows(strike_windows)   # reuse the same string you passed to the script

def which_window(a):
    for i, w in enumerate(wins):
        if in_windows(a, [w]):         # reuse your in_windows()
            return i
    return None

cnt = Counter(which_window(fp['strike']) for fp in full_stats['all_fault_params'])
total = sum(cnt.values())

print("\nStrike counts per window (full):")
for i, (lo, hi) in enumerate(wins):
    n = cnt.get(i, 0)
    pct = 100.0 * n / total if total else 0.0
    print(f"  window {i}: {lo:.1f}–{hi:.1f}° → {n} ({pct:.1f}%)")
print("  total faults counted:", total)

In [ ]:
for split in ("train", "validation"):
    path = os.path.join(STATS_DIR, f"statistics_{split}.json")
    if not os.path.exists(path):
        continue
    with open(path) as f:
        s = json.load(f)
    cnt = Counter(which_window(fp['strike']) for fp in s['all_fault_params'])
    total = sum(cnt.values())
    print(f"\n[{split}] strike counts per window:")
    for i, (lo, hi) in enumerate(wins):
        n = cnt.get(i, 0)
        pct = 100.0 * n / total if total else 0.0
        print(f"  window {i}: {lo:.1f}–{hi:.1f}° → {n} ({pct:.1f}%)")


In [ ]:
mode_used = "equal"  # or "span"
if mode_used == "equal":
    expected = [1/len(wins)] * len(wins)
else:
    spans = [((hi - lo) % 360) or 360 for lo, hi in wins]
    s = sum(spans)
    expected = [sp/s for sp in spans]
print("\nExpected % per window:", [round(100*x, 1) for x in expected])


In [ ]:
allowed = parse_windows(strike_windows)
bad = [fp['strike'] for fp in full_stats['all_fault_params']
       if not in_windows(fp['strike'], allowed)]
print("outside windows:", len(bad))


In [ ]:
# Load all statistics: full, prediction, train, and validation
stats_files = {
    "full": os.path.join(STATS_DIR, "statistics_full.json"),
    "train": os.path.join(STATS_DIR, "statistics_train.json"),
    "validation": os.path.join(STATS_DIR, "statistics_validation.json")
}

all_stats_data = {}
for split, path in stats_files.items():
    if os.path.exists(path):
        try:
            with open(path, 'r') as f:
                all_stats_data[split] = json.load(f)
            print(f"Loaded statistics for '{split}' from {path}")
        except Exception as e:
            print(f"Error loading statistics for '{split}': {e}")
            all_stats_data[split] = None
    else:
        print(f"Statistics file not found for '{split}': {path}")
        all_stats_data[split] = None

In [ ]:


# ─── 3) Display Statistics and Plots for All Splits Individually ──────────────────────────────

# palette for stacked plots
normal_colour  = (0.50, 0.00, 0.50)   # purple  (Normal)
inverse_colour = (0.00, 0.80, 0.00)   # green   (Inverse)
    
for split_name in ['full', 'train', 'validation']:
    stats = all_stats_data.get(split_name)
    print(f"\n--- Displaying Statistics and Plots for {split_name} data ---")

    if not stats or 'cube_level_params' not in stats or 'all_fault_params' not in stats:
        print(f"No statistics available for '{split_name}' split.")
        continue

    # Extract parameters for the current split
    cube_level_params = stats['cube_level_params']
    all_fault_params  = stats['all_fault_params']

    # Create a summary DataFrame
    df = pd.DataFrame(cube_level_params)
    print("\nCube Generation Parameter Summary:")
    print(df.describe())

    # ------------------------------------------------------------------
    # NEW ─ gather all mask cubes for pixel-class statistics
    # ------------------------------------------------------------------
    mask_cubes = []
    splits_to_scan = ['train', 'validation'] if split_name == 'full' else [split_name]
    for split_dir in splits_to_scan:
        mdir = os.path.join(base_out, split_dir, "fault")
        if not os.path.isdir(mdir):
            continue
        for fname in sorted(os.listdir(mdir)):
            if not fname.endswith((".npy", ".dat")):
                continue
            fpath = os.path.join(mdir, fname)
            if fname.endswith(".npy"):
                mask_cubes.append(np.load(fpath))
            else:  # .dat
                mask_cubes.append(np.fromfile(fpath, dtype=np.uint8)
                                  .reshape((cube_size,)*3))

    overall_pct, mean_pct = count_pixels(mask_cubes, mask_mode)

    print("\nPixel-class distribution – OVERALL (% of all voxels):")
    for k, v in overall_pct.items():
        print(f"  {k:>10}: {v:6.2f} %")
    print("Pixel-class distribution – MEAN PER CUBE:")
    for k, v in mean_pct.items():
        print(f"  {k:>10}: {v:6.2f} %")

    # --- Generate and Display Plots ---
    if not all_fault_params:
        print("\nNo fault parameters to plot for this split.")
        continue

    # Extract fault-specific data for plotting
    strikes               = [f['strike'] for f in all_fault_params]
    dips                  = [f['dip'] for f in all_fault_params]
    displacements = [f['applied_disp_signed'] for f in all_fault_params]
    vertical_displacements= [f['vertical_disp_component'] for f in all_fault_params]
    noise_sigmas          = [p.get('noise_sigma')         for p in cube_level_params]

    # Create a figure with a grid of subplots  (NOW 4×2)
    fig = plt.figure(figsize=(20, 22))
    fig.suptitle(f"{split_name.capitalize()} Dataset Statistics", fontsize=26)
    gs = fig.add_gridspec(4, 2, height_ratios=[1, 1, 1, 0.7])

    # Plot Fault Counts
    ax1 = fig.add_subplot(gs[0, 0])

    if mask_mode == 0:
        # binary → one bar
        ax1.bar(['Total'],
                [len(all_fault_params)],
                color=['skyblue'])
    else:
        # multiclass → two bars (Normal, Inverse)
        normal_count  = sum(1 for f in all_fault_params if f['fault_type'] == 'Normal')
        inverse_count = sum(1 for f in all_fault_params if f['fault_type'] == 'Inverse')
        ax1.bar(['Normal', 'Inverse'],
                [normal_count, inverse_count],
                color=[normal_colour, inverse_colour])

    ax1.set_title('Fault Type Distribution')
    ax1.set_ylabel('Number of Faults')

    # Plot Strike Rose Diagram
    ax2 = fig.add_subplot(gs[0, 1], projection='polar')
    if strikes:
        if mask_mode == 1:
            # --- separate angles
            strikes_normal  = np.radians([f['strike'] for f in all_fault_params if f['fault_type'] == 'Normal'])
            strikes_inverse = np.radians([f['strike'] for f in all_fault_params if f['fault_type'] == 'Inverse'])

            num_bins = 18
            bins_rad = np.linspace(0, 2*np.pi, num_bins + 1)
            cnt_norm , _ = np.histogram(strikes_normal , bins=bins_rad)
            cnt_inv  , _ = np.histogram(strikes_inverse, bins=bins_rad)

            centres = bins_rad[:-1] + np.diff(bins_rad)/2
            width   = np.diff(bins_rad)[0]

            # first ring = Normal
            ax2.bar(centres, cnt_norm,
                    width=width,
                    bottom=0.0,
                    color=normal_colour,
                    edgecolor='black',
                    alpha=0.8,
                    label='Normal')
            # stacked on top = Inverse
            ax2.bar(centres, cnt_inv,
                    width=width,
                    bottom=cnt_norm,
                    color=inverse_colour,
                    edgecolor='black',
                    alpha=0.8,
                    label='Inverse')

            ax2.legend(loc='lower left', bbox_to_anchor=(1.05, 0.0))
            ax2.set_title('Fault Strike Angles', va='bottom', y=1.1)
            ax2.set_theta_zero_location("N")  # 0° at North
            ax2.set_theta_direction(-1)       # clockwise

        else:
            # Explicit 0–360° rose (no 180° wrapping)
            angles = np.deg2rad(np.mod(strikes, 360.0))
            num_bins = 18
            bins_rad = np.linspace(0, 2*np.pi, num_bins + 1)
            cnt, _ = np.histogram(angles, bins=bins_rad)

            centres = bins_rad[:-1] + np.diff(bins_rad)/2
            width = np.diff(bins_rad)[0]

            ax2.bar(centres, cnt, width=width, bottom=0.0, edgecolor='black', alpha=0.9)
            ax2.set_title('Fault Strike Angles', va='bottom', y=1.1)
            ax2.set_theta_zero_location("N")  # 0° at North
            ax2.set_theta_direction(-1)       # clockwise


    # Plot Dip Histogram
    ax3 = fig.add_subplot(gs[1, 0])

    if dips:
        if mask_mode == 1:
            dips_normal  = [f['dip'] for f in all_fault_params if f['fault_type'] == 'Normal']
            dips_inverse = [f['dip'] for f in all_fault_params if f['fault_type'] == 'Inverse']
            ax3.hist([dips_normal, dips_inverse],
                     bins='auto',
                     stacked=True,
                     color=[normal_colour, inverse_colour],
                     label=['Normal', 'Inverse'],
                     edgecolor='black')
            ax3.legend()
        else:
            plot_histogram(dips, ax=ax3, title='Fault Dip Angles',
                           xlabel='Dip (degrees)')
        ax3.set_title('Fault Dip Angles')
        ax3.set_xlabel('Dip (degrees)')

    # Plot Displacement Histogram
    ax4 = fig.add_subplot(gs[1, 1])
    if displacements:
        if mask_mode == 1:
            disp_normal  = [f['applied_disp_signed'] for f in all_fault_params if f['fault_type'] == 'Normal']
            disp_inverse = [f['applied_disp_signed'] for f in all_fault_params if f['fault_type'] == 'Inverse']
            ax4.hist([disp_normal, disp_inverse],
                     bins='auto',
                     stacked=True,
                     color=[normal_colour, inverse_colour],
                     label=['Normal', 'Inverse'],
                     edgecolor='black')
            ax4.legend()
        else:
            plot_histogram(displacements, ax=ax4, title='Applied Displacement',
                           xlabel='Displacement (grid units)')
        ax4.set_title('Applied Displacement')
        ax4.set_xlabel('Displacement (grid units)')

    # Plot Vertical Displacement Histogram
    ax5 = fig.add_subplot(gs[2, 0])
    if vertical_displacements:
        if mask_mode == 1:
            vdisp_normal  = [f['vertical_disp_component'] for f in all_fault_params if f['fault_type'] == 'Normal']
            vdisp_inverse = [f['vertical_disp_component'] for f in all_fault_params if f['fault_type'] == 'Inverse']
            ax5.hist([vdisp_normal, vdisp_inverse],
                     bins='auto',
                     stacked=True,
                     color=[normal_colour, inverse_colour],
                     label=['Normal', 'Inverse'],
                     edgecolor='black')
            ax5.legend()
        else:
            plot_histogram(vertical_displacements, ax=ax5,
                           title='Vertical Displacement Component',
                           xlabel='Vertical Displacement (grid units)')
        ax5.set_title('Vertical Displacement Component')
        ax5.set_xlabel('Vertical Displacement (grid units)')

    # Plot Noise Sigma Histogram
    ax6 = fig.add_subplot(gs[2, 1])
    if any(n is not None for n in noise_sigmas):
        plot_histogram([n for n in noise_sigmas if n is not None],
                       ax=ax6, title='Seismic Noise Sigma',
                       xlabel='Noise Sigma')

    # ------------------------------------------------------------------
    # NEW subplot: pixel-class distribution
    # ------------------------------------------------------------------
    ax7 = fig.add_subplot(gs[3, :])
    categories = list(overall_pct.keys())

    if mask_mode == 0:
        colours = ['dimgrey', 'skyblue'][:len(categories)]
    else:
        colour_map = {
            'no_fault': 'dimgrey',
            'normal'  : (0.50, 0.00, 0.50),  # purple
            'inverse' : (0.00, 0.80, 0.00)   # green
        }
        colours = [colour_map[c] for c in categories]

    ax7.bar(categories,
            [overall_pct[k] for k in categories],
            color=colours)
    ax7.set_ylim(0, 100)
    ax7.set_title('Pixel-Class Distribution (% of all voxels)')
    ax7.set_ylabel('Percentage')
    for i, k in enumerate(categories):
        ax7.text(i, overall_pct[k] + 1, f"{overall_pct[k]:.1f}%", ha='center')

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    img_path = os.path.join(IMAGE_DIR, f"{split_name}_dataset_stats.png")
    fig.savefig(img_path, dpi=300)
    print(f"Saved figure → {img_path}")
    plt.show()


---

In [ ]:
# Prepare colormaps for sample slices
bw_cmap             = "gray"
overlay_bw_cmap     = ListedColormap([(0,0,0,0), (1,1,1,1)]) # Transparent to White

mask_color_alone_cmap   = ListedColormap([
    (0.00, 0.00, 0.00, 1.0),  # 0 → black
    (0.50, 0.00, 0.50, 1.0),  # 1 → purple (Normal)
    (0.00, 0.80, 0.00, 1.0)   # 2 → green (Inverse)
])
mask_color_overlay_cmap = ListedColormap([
    (0,0,0,0),           # 0 → transparent
    (0.50,0.00,0.50,1),  # 1 → purple (Normal)
    (0.00,0.80,0.00,1)   # 2 → green (Inverse)
])

# Select colormap depending on mask_mode
if mask_mode == 0:
    mask_alone_cmap_slice   = bw_cmap
    mask_overlay_cmap_slice = overlay_bw_cmap
else:
    mask_alone_cmap_slice   = mask_color_alone_cmap
    mask_overlay_cmap_slice = mask_color_overlay_cmap

# Transparency factor for overlays
overlay_alpha = 1.0

---

---

### 3D Visualization

In [ ]:
# # Normalization helper
# def normalize(v):
#     return ((v - v.min()) / (v.max() - v.min()) * 255).astype(np.uint8)

In [ ]:
# # Fault plotting function
# def plot_fault_points(fig, slice_mask, axis_index, slice_type, color, label, nx, ny, nz):
#     if slice_type == "inline":
#         ks, js = np.where(slice_mask)
#         fig.add_trace(go.Scatter3d(
#             x=np.full_like(ks, axis_index),
#             y=js,
#             z=nz - 1 - ks,
#             mode='markers',
#             marker=dict(color=color, size=2, opacity=1),
#             name=label
#         ))
#     elif slice_type == "crossline":
#         ks, is_ = np.where(slice_mask)
#         fig.add_trace(go.Scatter3d(
#             x=is_,
#             y=np.full_like(is_, axis_index),
#             z=nz - 1 - ks,
#             mode='markers',
#             marker=dict(color=color, size=2, opacity=1),
#             name=label
#         ))
#     elif slice_type == "timeslice":
#         is_, js = np.where(slice_mask)
#         fig.add_trace(go.Scatter3d(
#             x=is_,
#             y=js,
#             z=np.full_like(is_, axis_index),
#             mode='markers',
#             marker=dict(color=color, size=2, opacity=1),
#             name=label
#         ))

In [ ]:
# # --------------------------------------------------------------------------
# # 3-D preview of one cube + fault-by-fault table
# # --------------------------------------------------------------------------

# def show_fault_cube(split:str, idx:int|None=None, *, seed:int|None=None):
#     """
#     Visualise a random (or chosen) cube and list every individual fault.

#     Parameters
#     ----------
#     split : {"train", "validation"}
#     idx   : int | None
#         Exact cube-index (global index used for the file-name).  If *None*
#         a cube is selected at random **but only among cubes that contain ≥ 1
#         faults**.
#     seed  : int | None
#         Fixes RNG state so the *same* random cube is chosen every run.
#     """

#     if seed is not None:
#         random.seed(seed)

#     # ------------------------------------------------------------------ paths
#     seis_dir  = os.path.join(base_out, split, "seis")
#     mask_dir  = os.path.join(base_out, split, "fault")
#     stats_f   = os.path.join(STATS_DIR, f"statistics_{split}.json")

#     if not (os.path.isdir(seis_dir) and os.path.isdir(mask_dir)):
#         print(f"[{split}]   output folders not found.")
#         return
#     if not os.path.exists(stats_f):
#         print(f"[{split}]   statistics file missing: {stats_f}")
#         return

#     # ---------------------------------------------------------------- stats –
#     with open(stats_f, "r") as f:
#         stats_json = json.load(f)
#     # map cube-id → list[dict]  (one dict per fault)
#     by_cube = {}
#     for fp in stats_json["all_fault_params"]:
#         by_cube.setdefault(fp["cube_id"], []).append(fp)

#     # all cubes that *actually* have faults
#     cubes_with_faults = [cid for cid, lst in by_cube.items() if len(lst) > 0]

#     # ---------------------------------------------------------------- pick id
#     if idx is None:
#         if not cubes_with_faults:
#             print(f"[{split}]   no faults at all in this split.")
#             return
#         idx = random.choice(cubes_with_faults)
#     else:
#         # sanity-check requested id
#         if idx not in by_cube or len(by_cube[idx]) == 0:
#             print(f"[{split}]   cube {idx} exists but has no faults.")
#             return

#     fname = f"{idx}.{output_format}"
#     if not (os.path.exists(os.path.join(seis_dir, fname)) and
#             os.path.exists(os.path.join(mask_dir, fname))):
#         print(f"[{split}]   data files for cube {idx} not found.")
#         return

#     print(f"[{split}] cube {idx}  →  {fname}")

#     # ---------------------------------------------------------------- load vol
#     if fname.endswith(".npy"):
#         seismic = np.load(os.path.join(seis_dir,  fname))
#         mask    = np.load(os.path.join(mask_dir,  fname))
#     else:  # ".dat"
#         shape  = (cube_size,)*3
#         seismic = np.fromfile(os.path.join(seis_dir,  fname),
#                               dtype=np.float32).reshape(shape)
#         mask    = np.fromfile(os.path.join(mask_dir,  fname),
#                               dtype=np.uint8 ).reshape(shape)

#     nx, ny, nz = seismic.shape
#     inline,  inline_m   = seismic[nx//2,:,:].T, mask[nx//2,:,:].T
#     crossline, cross_m  = seismic[:,ny//2,:].T, mask[:,ny//2,:].T
#     timeslice, times_m  = seismic[:,:,nz//2]  , mask[:,:,nz//2]

#     # normalise slices for Surface display
#     norm = lambda v: ((v-v.min())/(v.max()-v.min()+1e-12)*255).astype(np.uint8)
#     inline_n, cross_n, time_n = map(norm, (inline, crossline, timeslice))

#     # ---------------------------------------------------------------- figure
#     fig = make_subplots(rows=1, cols=2,
#                         specs=[[{'type':'scene'}, {'type':'table'}]],
#                         column_widths=[0.75, 0.25],
#                         horizontal_spacing=0.04)

#     # seismic surfaces
#     fig.add_trace(go.Surface(z=np.tile(np.arange(nz)[::-1], (ny,1)).T,
#                              x=np.full((nz,ny), nx//2),
#                              y=np.tile(np.arange(ny), (nz,1)),
#                              surfacecolor=inline_n, colorscale="Gray",
#                              showscale=False),
#                   row=1,col=1)
#     fig.add_trace(go.Surface(z=np.tile(np.arange(nz)[::-1], (nx,1)).T,
#                              x=np.tile(np.arange(nx), (nz,1)),
#                              y=np.full((nz,nx), ny//2),
#                              surfacecolor=cross_n, colorscale="Gray",
#                              showscale=False),
#                   row=1,col=1)
#     fig.add_trace(go.Surface(z=np.full((ny,nx), nz//2),
#                              x=np.tile(np.arange(nx), (ny,1)),
#                              y=np.tile(np.arange(ny).reshape(-1,1), (1,nx)),
#                              surfacecolor=np.rot90(time_n, k=-1),
#                              colorscale="Gray", showscale=False),
#                   row=1,col=1)

#     # fault dots
#     if mask_mode == 0:               # binary
#         plot_fault_points(fig, inline_m==1 , nx//2, "inline"   ,"white","Inline", nx,ny,nz)
#         plot_fault_points(fig, cross_m==1  , ny//2, "crossline","white","Cross",  nx,ny,nz)
#         plot_fault_points(fig, times_m==1  , nz//2, "timeslice","white","T-slice",nx,ny,nz)
#     else:                            # multi-class
#         colours = {1:"green", 2:"purple"}
#         for cls,label in [(1,"Normal"), (2,"Inverse")]:
#             plot_fault_points(fig, inline_m==cls , nx//2,"inline",   colours[cls],f"Inl {label}", nx,ny,nz)
#             plot_fault_points(fig, cross_m==cls  , ny//2,"crossline",colours[cls],f"Xl  {label}", nx,ny,nz)
#             plot_fault_points(fig, times_m==cls  , nz//2,"timeslice",colours[cls],f"Tsl {label}", nx,ny,nz)

#     # ---------------------------------------------------------------- table
#     faults = by_cube[idx]                      # list[dict], already non-empty
#     dips     = [f["dip"]                 for f in faults]
#     strikes  = [f["strike"]              for f in faults]
#     slips    = [f["applied_disp_signed"] for f in faults]
#     if mask_mode == 1:
#         ftypes = [f["fault_type"] or "?" for f in faults]

#     col_vals = [["#",
#                  "Dip (°)",
#                  "Strike (°)",
#                  "Slip (vox)"] +
#                 (["Type"] if mask_mode == 1 else []),
#                 list(range(1, len(faults)+1)),
#                 [f"{d:5.1f}" for d in dips],
#                 [f"{s:6.1f}" for s in strikes],
#                 [f"{sl:6.1f}"for sl in slips]]
#     if mask_mode == 1:
#         col_vals.append(ftypes)

#     header = dict(values=col_vals[0], align="center", fill_color="#d3d3d3")
#     cells  = dict(values=col_vals[1:], align="center")
#     fig.add_trace(go.Table(header=header, cells=cells), row=1, col=2)

#     # ---------------------------------------------------------------- layout
#     fig.update_layout(title=f"{split.upper()} cube {idx}",
#                       scene=dict(aspectmode="data",
#                                  xaxis_visible=False,
#                                  yaxis_visible=False,
#                                  zaxis_visible=False),
#                       margin=dict(l=0, r=0, t=40, b=0))
#     display(fig)


In [ ]:
# show_fault_cube("train")          # random training cube or show_fault_cube("train", idx=42) # specific cube if you know the index

In [ ]:
# show_fault_cube("validation") # random validation cube

In [ ]:
# # --- CIGVis imports ---


# # Optional: axis & order tweaks if your data needs it
# # config.set_order(line_first=True)           # keep default (x=inline first)
# # config.set_axis_reversed(x=False, y=False, z=True)  # flip if your z is downwards, etc.

# def view_cube_with_cigvis(base_out, stats_dir, split="train", idx=None, pos=None,
#                           clim_pct=(1, 99), mask_mode=0, output_format="npy"):
#     """
#     Visualize one cube with CIGVis (Plotly). Overlays binary or multi-class masks.
#     """
#     seis_dir  = os.path.join(base_out, split, "seis")
#     mask_dir  = os.path.join(base_out, split, "fault")
#     stats_f   = os.path.join(stats_dir, f"statistics_{split}.json")

#     # Pick a cube that actually has faults if idx not provided
#     if idx is None and os.path.exists(stats_f):
#         with open(stats_f, "r") as f:
#             s = json.load(f)
#         by_cube = {}
#         for fp in s["all_fault_params"]:
#             by_cube.setdefault(fp["cube_id"], []).append(fp)
#         candidates = [cid for cid, lst in by_cube.items() if len(lst) > 0]
#         if not candidates:
#             raise RuntimeError(f"No cubes with faults found in split={split}.")
#         idx = random.choice(candidates)

#     fname = f"{idx}.{output_format}"
#     seis_p = os.path.join(seis_dir, fname)
#     mask_p = os.path.join(mask_dir, fname)
#     if not (os.path.exists(seis_p) and os.path.exists(mask_p)):
#         raise FileNotFoundError(f"Missing files for cube {idx} in {split}")

#     # Load volumes (CIGVis expects (ni, nx, nt) = (inline, crossline, time))
#     if fname.endswith(".npy"):
#         seismic = np.load(seis_p).astype(np.float32)
#         mask    = np.load(mask_p)
#     else:  # ".dat"
#         # adjust dtype/shape if your writer differs
#         shape = (seismic.shape)  # replace with your known cube_size tuple if needed
#         seismic = np.fromfile(seis_p, np.float32).reshape(shape)
#         mask    = np.fromfile(mask_p,  np.uint8 ).reshape(shape)

#     # Pick slice positions & contrast
#     ni, nx, nt = seismic.shape
#     if pos is None:
#         pos = [ni//2, nx//2, nt//2]
#     vmin, vmax = np.percentile(seismic, clim_pct)

#     # ---- discrete cmap for mask (0=background transparent; 1/2 are classes)
#     values = sorted(np.unique(mask.astype(int)).tolist())  # e.g. [0,1] or [0,1,2]
#     color_map = {
#         0: (1, 1, 1),          # background (will be transparent)
#         1: (0.50, 0.00, 0.50), # Normal (purple)
#         2: (0.00, 0.80, 0.00), # Inverse (green)
#     }
#     colors   = [color_map.get(v, (1.0, 0.5, 0.0)) for v in values]
#     fg_cmap  = colormap.custom_disc_cmap(values, colors)
#     fg_cmap  = colormap.set_alpha_except_min(fg_cmap, 0.9)  # make value==min(values) transparent

#     # ---- build overlay nodes (seismic as background, mask as foreground)
#     nodes = create_overlay(
#         bg_volume=seismic,
#         fg_volume=mask.astype(int),
#         pos=pos,
#         bg_clim=[float(vmin), float(vmax)],
#         fg_clim=[min(values), max(values)],
#         bg_cmap="Petrel",
#         fg_cmap=fg_cmap,
#         show_cbar=True,        # show legend; set cbar_type="fg" to show mask colors
#         cbar_type="fg"
#     )

#     plot3D(nodes)

In [ ]:
# view_cube_with_cigvis(
#     base_out=DATA_DIR,
#     stats_dir=STATS_DIR,
#     split="train",      # or "validation"
#     idx=None,           # or a specific cube id, e.g. 42
#     mask_mode=mask_mode,
#     output_format=output_format
# )


In [ ]:
# from cigvis.plotlyplot import create_slices, plot3D
# from cigvis import colormap

# # bg: seismic
# nodes = create_slices(
#     seismic, pos=pos,
#     clim=[float(vmin), float(vmax)],
#     cmap="Petrel", show_cbar=True
# )

# # fg: mask (discrete cmap; make background transparent)
# values  = sorted(np.unique(mask.astype(int)).tolist())
# color_map = {0:(1,1,1), 1:(0.50,0.00,0.50), 2:(0.00,0.80,0.00)}
# colors  = [color_map.get(v, (1.0,0.5,0.0)) for v in values]
# fg_cmap = colormap.custom_disc_cmap(values, colors)
# fg_cmap = colormap.set_alpha_except_min(fg_cmap, 0.9)

# nodes += create_slices(
#     mask.astype(int), pos=pos,
#     clim=[min(values), max(values)],
#     cmap=fg_cmap,
#     interpolation="nearest",   # keep classes crisp
#     show_cbar=True
# )

# plot3D(nodes)


In [ ]:
# # CIGVis + Plotly: grayscale seismic, red mask, interactive slice sliders

# import numpy as np
# import cigvis.plotlyplot as cgp
# from cigvis import colormap, config
# from ipywidgets import IntSlider, HBox, VBox, interactive_output
# from IPython.display import display

# # --- your files ---
# seis_p = "/home/roderickperez/DS_PROJECTS/faultSeg/generateSynthData/data/train/seis/0.npy"
# mask_p = "/home/roderickperez/DS_PROJECTS/faultSeg/generateSynthData/data/train/fault/0.npy"

# # --- load volumes ---
# seismic = np.load(seis_p).astype(np.float32)
# mask    = np.load(mask_p).astype(np.int32)
# assert seismic.shape == mask.shape, f"Shape mismatch: {seismic.shape} vs {mask.shape}"

# ni, nx, nt = seismic.shape
# # If time/depth looks upside-down in your view, uncomment:
# # config.set_axis_reversed(z=True)

# # Contrast window for grayscale background
# vmin, vmax = np.percentile(seismic, (1, 99))

# # --- red discrete cmap for mask (0=transparent; all nonzero=red) ---
# values = np.unique(mask).astype(int)
# cols = []
# for v in values:
#     if v == values.min():    # assume min==0 is background
#         cols.append((1.0, 1.0, 1.0, 0.0))  # fully transparent
#     else:
#         cols.append((1.0, 0.0, 0.0, 0.9))  # red with opacity
# fg_cmap = colormap.custom_disc_cmap(values.tolist(), cols)
# # Keep min value transparent, others ~0.9 opacity
# fg_cmap = colormap.set_alpha_except_min(fg_cmap, 0.9)

# def build_nodes(i_pos, x_pos, t_pos):
#     pos = [i_pos, x_pos, t_pos]
#     if hasattr(cgp, "create_overlay"):
#         # Overlay mask on grayscale seismic
#         nodes = cgp.create_overlay(
#             bg_volume=seismic,
#             fg_volume=mask,
#             pos=pos,
#             bg_clim=[float(vmin), float(vmax)],
#             fg_clim=[int(values.min()), int(values.max())],
#             bg_cmap="gray",          # <-- grayscale seismic
#             fg_cmap=fg_cmap,         # <-- red mask
#             show_cbar=False          # avoid Plotly colorbar arg mismatch
#         )
#     else:
#         # Fallback: stack slices (still grayscale + red)
#         nodes = cgp.create_slices(
#             seismic, pos=pos, clim=[float(vmin), float(vmax)],
#             cmap="gray", show_cbar=False
#         )
#         nodes += cgp.create_slices(
#             mask, pos=pos,
#             clim=[int(values.min()), int(values.max())],
#             cmap=fg_cmap, interpolation="nearest",
#             show_cbar=False
#         )
#     return nodes

# # --- interactive sliders (update on release) ---
# w_i = IntSlider(min=0, max=ni-1, step=1, value=ni//2, description="Inline",  continuous_update=False)
# w_x = IntSlider(min=0, max=nx-1, step=1, value=nx//2, description="Xline",  continuous_update=False)
# w_t = IntSlider(min=0, max=nt-1, step=1, value=nt//2, description="Time",    continuous_update=False)

# def update(i, x, t):
#     nodes = build_nodes(i, x, t)
#     cgp.plot3D(nodes)  # renders the current slices

# out = interactive_output(update, {"i": w_i, "x": w_x, "t": w_t})
# display(VBox([HBox([w_i, w_x, w_t]), out]))


In [ ]:
# from ipywidgets import IntSlider, Play, Button, HBox, VBox, interactive_output, jslink

# # sliders (live updates while dragging)
# w_i = IntSlider(min=0, max=ni-1, step=1, value=ni//2, description="Inline",   continuous_update=True)
# w_x = IntSlider(min=0, max=nx-1, step=1, value=nx//2, description="Xline",    continuous_update=True)
# w_t = IntSlider(min=0, max=nt-1, step=1, value=nt//2, description="Time",     continuous_update=True)

# # tiny helpers to step +/- with buttons
# def step(slider, d): 
#     slider.value = max(slider.min, min(slider.max, slider.value + d))

# btn_i_minus = Button(description="Inline −")
# btn_i_plus  = Button(description="Inline +")
# btn_x_minus = Button(description="Xline −")
# btn_x_plus  = Button(description="Xline +")
# btn_t_minus = Button(description="Time −")
# btn_t_plus  = Button(description="Time +")

# btn_i_minus.on_click(lambda _: step(w_i, -1))
# btn_i_plus .on_click(lambda _: step(w_i, +1))
# btn_x_minus.on_click(lambda _: step(w_x, -1))
# btn_x_plus .on_click(lambda _: step(w_x, +1))
# btn_t_minus.on_click(lambda _: step(w_t, -1))
# btn_t_plus .on_click(lambda _: step(w_t, +1))

# # Play widget to auto-scan time
# play = Play(interval=60, value=w_t.value, min=w_t.min, max=w_t.max, step=1, description="▶")
# jslink((play, 'value'), (w_t, 'value'))  # link playhead to time slider

# def update(i, x, t):
#     nodes = build_nodes(i, x, t)  # your function from earlier
#     cgp.plot3D(nodes)

# out = interactive_output(update, {"i": w_i, "x": w_x, "t": w_t})
# display(VBox([
#     HBox([w_i, btn_i_minus, btn_i_plus]),
#     HBox([w_x, btn_x_minus, btn_x_plus]),
#     HBox([w_t, btn_t_minus, btn_t_plus, play]),
#     out
# ]))


In [ ]:
# --- CIGVis viewer (grayscale seismic + red mask, with sliders) ---
import os, json, random
import numpy as np
import cigvis.plotlyplot as cgp
from cigvis import colormap, config
from ipywidgets import IntSlider, Play, Button, HBox, VBox, interactive_output, jslink
from IPython.display import display

BASE_OUT  = DATA_DIR    # already defined earlier
STATS_DIR = STATS_DIR   # already defined earlier
FMT       = output_format  # "npy" or "dat"

def choose_idx_with_faults(split="train"):
    stats_f = os.path.join(STATS_DIR, f"statistics_{split}.json")
    with open(stats_f, "r") as f:
        s = json.load(f)
    by_cube = {}
    for fp in s["all_fault_params"]:
        by_cube.setdefault(fp["cube_id"], []).append(fp)
    cands = [cid for cid, lst in by_cube.items() if lst]
    if not cands:
        raise RuntimeError(f"No cubes with faults in split={split}")
    return random.choice(cands)

def load_volumes(split, idx, fmt=FMT, cube_size=(cube_size,)*3):
    fn = f"{idx}.{fmt}"
    seis_p = os.path.join(BASE_OUT, split, "seis", fn)
    mask_p = os.path.join(BASE_OUT, split, "fault", fn)
    if not (os.path.exists(seis_p) and os.path.exists(mask_p)):
        raise FileNotFoundError(f"Missing files for cube {idx} in {split}")
    if fmt == "npy":
        seismic = np.load(seis_p).astype(np.float32)
        mask    = np.load(mask_p).astype(np.int32)
    else:
        seismic = np.fromfile(seis_p, np.float32).reshape(cube_size)
        mask    = np.fromfile(mask_p,  np.uint8 ).reshape(cube_size).astype(np.int32)
    return seismic, mask

# Pick a cube that actually has faults
split   = "train"            # or "validation"
idx     = choose_idx_with_faults(split)
seismic, mask = load_volumes(split, idx)

# Contrast & mask mapping
vmin, vmax = np.percentile(seismic, (1, 99))
mask_bin   = (mask > 0).astype(np.int32)     # any nonzero → 1 (single red class)
values     = [0, 1]
cols       = [(1.0, 1.0, 1.0, 0.0), (1.0, 0.0, 0.0, 0.9)]  # 0 transparent, 1 red
fg_cmap    = colormap.custom_disc_cmap(values, cols)
fg_cmap    = colormap.set_alpha_except_min(fg_cmap, 0.9)

# Optional: flip Z if your time/depth is downwards
# config.set_axis_reversed(z=True)

ni, nx, nt = seismic.shape
def build_nodes(i, x, t):
    pos = [int(i), int(x), int(t)]
    if hasattr(cgp, "create_overlay"):
        nodes = cgp.create_overlay(
            bg_volume=seismic,
            fg_volume=mask_bin,
            pos=pos,
            bg_clim=[float(vmin), float(vmax)],
            fg_clim=[0, 1],
            bg_cmap="gray",          # grayscale seismic
            fg_cmap=fg_cmap,         # red mask
            show_cbar=False
        )
    else:
        # Fallback: stacked slices
        nodes  = cgp.create_slices(seismic, pos=pos, clim=[float(vmin), float(vmax)],
                                   cmap="gray", show_cbar=False)
        nodes += cgp.create_slices(mask_bin, pos=pos, clim=[0, 1],
                                   cmap=fg_cmap, interpolation="nearest", show_cbar=False)
    return nodes

# --- Sliders + small step buttons + time play ---
w_i = IntSlider(min=0, max=ni-1, step=1, value=ni//2, description="Inline",   continuous_update=True)
w_x = IntSlider(min=0, max=nx-1, step=1, value=nx//2, description="Xline",    continuous_update=True)
w_t = IntSlider(min=0, max=nt-1, step=1, value=nt//2, description="Time",     continuous_update=True)

def step(slider, d):
    slider.value = max(slider.min, min(slider.max, slider.value + d))

btn_i_minus = Button(description="Inline −"); btn_i_minus.on_click(lambda _: step(w_i, -1))
btn_i_plus  = Button(description="Inline +"); btn_i_plus .on_click(lambda _: step(w_i, +1))
btn_x_minus = Button(description="Xline −");  btn_x_minus.on_click(lambda _: step(w_x, -1))
btn_x_plus  = Button(description="Xline +");  btn_x_plus .on_click(lambda _: step(w_x, +1))
btn_t_minus = Button(description="Time −");   btn_t_minus.on_click(lambda _: step(w_t, -1))
btn_t_plus  = Button(description="Time +");   btn_t_plus .on_click(lambda _: step(w_t, +1))

play = Play(interval=60, value=w_t.value, min=w_t.min, max=w_t.max, step=1, description="▶")
jslink((play, 'value'), (w_t, 'value'))

def update(i, x, t):
    nodes = build_nodes(i, x, t)
    cgp.plot3D(nodes)

out = interactive_output(update, {"i": w_i, "x": w_x, "t": w_t})
display(VBox([
    HBox([w_i, btn_i_minus, btn_i_plus]),
    HBox([w_x, btn_x_minus, btn_x_plus]),
    HBox([w_t, btn_t_minus, btn_t_plus, play]),
    out
]))
print(f"Viewing split='{split}', cube id={idx}")
